In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

import torch
from torch import nn
from torch.nn.utils import weight_norm
import torch.optim as optim
import torch.nn.functional as F

from skorch import NeuralNetClassifier, NeuralNet
from skorch.callbacks import EpochScoring
from skorch.callbacks import LRScheduler, EarlyStopping
from torch.optim.lr_scheduler import ReduceLROnPlateau


import warnings

In [ ]:
df = pd.read_csv("../../data/tabular/train_fold.csv")

In [ ]:
cat_cols = [c for c in df.columns if 'feature' in c]
for col in cat_cols:
    df[col]= df[col].astype('category')
    
cat_szs = [max(df[col]) + 1 for col in cat_cols]
emb_szs = [(size, min(50, (size+1)//3)) for size in cat_szs]

In [ ]:

warnings.filterwarnings("ignore")

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(0)

feature_dictionary_size= 360
num_features = 75

def residual_block(in_features, out_features, p_drop, non_linear = nn.ReLU(), *args, **kwargs):
    return nn.Sequential(
        nn.Dropout(p = p_drop),
        weight_norm(nn.Linear(in_features, out_features)),
        non_linear
    )

class TPSResidual(nn.Module):
    def __init__(self, num_class = 9, emb_szs = None, dropout = 0.3, linear_nodes=32, linear_out=16, emb_output=4, num_block=3):
        super().__init__()
        self.num_block = num_block
        
        self.final_module_list = nn.ModuleList()

        #self.embedding = nn.Embedding(feature_dictionary_size, emb_output)
        self.embeds = nn.ModuleList([nn.Embedding(ni, nf) for ni,nf in emb_szs]) #type: torch.nn.modules.container.ModuleList
        self.n_emb = sum(e.embedding_dim for e in self.embeds) # n_emb = 17 , type: int

        self.flatten = nn.Flatten()

        self.linear = weight_norm(nn.Linear(self.n_emb, linear_nodes))
        #self.linear = weight_norm(nn.Linear(emb_output * num_features, linear_nodes ))

        for res_num in range(self.num_block):
            self.non_linear = nn.ELU() if res_num %2 else nn.ReLU()
            self.lin_out = linear_out if res_num == (self.num_block - 1) else linear_nodes
            self.final_module_list.append(residual_block( self.n_emb + (res_num + 1) * linear_nodes, 
                                self.lin_out, dropout, self.non_linear))
        self.out = nn.Linear(linear_out, num_class)

        # non-linearity - activation function
        self.selu = nn.SELU()
        self.dropout = nn.Dropout(p = dropout)

    def forward(self, x):
        # Embedding
        if self.n_emb != 0:
            x = [e(x[:,i]) for i,e in enumerate(self.embeds)] #take the embedding list and grab an embedding and pass in our single row of data.        
            x = torch.cat(x, 1) # concatenate it on dim 1 ## remeber that the len is the batch size
            x = self.dropout(x) # pass it through a dropout layer
        e = self.flatten(x)
        
        h1 = self.dropout(e)
        h1 = self.linear(h1)
        h1 = self.selu(h1)

        ri = torch.cat((e, h1), 1)
        for res_num in range(self.num_block):
            rx = self.final_module_list[res_num](ri)
            ri = torch.cat((ri, rx), 1)
        
        return F.softmax(self.out(rx), dim =-1)

lr_scheduler = LRScheduler(policy = ReduceLROnPlateau, monitor = 'valid_loss', mode = 'min', patience = 3, factor = 0.1, verbose = True)
early_stopping = EarlyStopping(monitor='valid_loss', patience = 10, threshold = 0.0001, threshold_mode='rel', lower_is_better=True)
 
net = NeuralNetClassifier(module=TPSResidual, module__emb_szs=emb_szs,
                          device = device, lr = 0.01, max_epochs = 50, 
                          callbacks = [lr_scheduler, early_stopping], batch_size=64
                         )

In [ ]:
kfold = 0

In [ ]:
train_df = df[df['kfold'] != kfold]
valid_df = df[df['kfold']==kfold]
lencoder = LabelEncoder()

X_train = train_df.drop(['id','target','kfold'],axis=1).values.astype('int64')
y_train = lencoder.fit_transform(train_df['target']).astype('int64')
y_valid = lencoder.transform(valid_df['target']).astype('int64')

net.fit(X_train, y_train)

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=TPSResidual(
    (final_module_list): ModuleList(
      (0): Sequential(
        (0): Dropout(p=0.3, inplace=False)
        (1): Linear(in_features=1565, out_features=32, bias=True)
        (2): ReLU()
      )
      (1): Sequential(
        (0): Dropout(p=0.3, inplace=False)
        (1): Linear(in_features=1597, out_features=32, bias=True)
        (2): ELU(alpha=1.0)
      )
      (2): Sequential(
        (0): Dropout(p=0.3, inplace=False)
        (1): Linear(in_features=1629, out_features=16, bias=True)
        (2): ReLU()
      )
    )
    (embeds): ModuleList(
      (0): Embedding(62, 21)
      (1): Embedding(52, 17)
      (2): Embedding(65, 22)
      (3): Embedding(71, 24)
      (4): Embedding(39, 13)
      (5): Embedding(77, 26)
      (6): Embedding(44, 15)
      (7): Embedding(31, 10)
      (8): Embedding(39, 13)
      (9): Embedding(73, 24)
      (10): Embedding(34, 11)
      (11): Embedding(47, 16)
      (1

In [ ]:
X_valid = valid_df.drop(['id','target','kfold'],axis=1).values.astype('int64')

In [ ]:
y_pred = net.predict(X_valid)

In [ ]:
from sklearn import metrics

In [ ]:
metrics.accuracy_score(y_valid, )